In [1]:
setwd('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1009. NC ENVRIOSCAN/P1009.3. Analyses/P1009.3.3. Sociodemographic Imputation/Input')
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1009. NC ENVRIOSCAN/P1009.3. Analyses/P1009.3.3. Sociodemographic Imputation/Output')
cur_date = "010323"

library(readxl)
library(tidyverse)
library(reshape2)
library(missForest)

# reading in files
acs_df = data.frame(read_excel("ACS_Data_121222.xlsx", sheet = 2))
# need to make some of the cols numeric
acs_df[,4:30] = apply(acs_df[,4:30], 2, as.numeric)
wildfire_hazard_df = data.frame(read_excel("Wildfire_Hazard_010323.xlsx", sheet = 2))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coe

In [2]:
head(acs_df)
head(wildfire_hazard_df)

,GEO_ID,Census_Tract,County,Sex_Ratio,Median_Age,Population_Under_18,Population_Over_18,Population_Over_65,Race_White,Race_Black,⋯,Poverty_Adult,Poverty_Elderly,Poverty_Male,Poverty_Female,Poverty_Black,Poverty_Asian,Poverty_Other_Race,Poverty_Mixed_Race,Poverty_Hispanic_Latino,Poverty_White
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,14000US37001020100,201.00,Alamance,87.2,37.3,21.9,78.1,15.1,61.0,28.5,⋯,20.8,14.6,26.4,28.0,54.8,0,45.3,0.0,31.3,13.9
2,14000US37001020200,202.00,Alamance,78.3,31.0,29.1,70.9,11.2,51.9,29.2,⋯,33.0,19.2,27.3,37.8,44.6,0,0.0,82.7,16.2,39.0
3,14000US37001020300,203.00,Alamance,77.8,40.7,27.1,72.9,18.9,45.2,28.9,⋯,17.3,22.0,18.0,28.3,21.0,NA,10.9,36.4,26.7,20.1
4,14000US37001020400,204.00,Alamance,78.1,30.5,31.4,68.6,10.1,19.6,59.0,⋯,26.8,14.0,30.0,33.3,19.9,NA,69.0,100.0,67.2,26.6
5,14000US37001020501,205.01,Alamance,92.7,35.1,23.3,76.7,15.6,76.1,13.3,⋯,12.2,7.1,12.3,14.8,18.9,0,0.0,53.1,25.7,10.0
6,14000US37001020502,205.02,Alamance,99.0,32.1,30.0,70.0,8.9,40.7,35.7,⋯,22.3,19.2,17.7,31.0,27.8,100,14.2,19.5,14.7,25.0


,GEO_ID,County,Census_Tract,Avg_ACRES,Wildfire_Hazard_Potential_Mean,Wildfire_Hazard_Potential_Quintile_Number,Wildfire_Hazard_Potential_Quintile_Percentage
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,14000US37141920201,Pender,9202.01,596842.6,715.5177,1,0-20%
2,14000US37141920401,Pender,9204.01,596842.6,715.5177,1,0-20%
3,14000US37141920402,Pender,9204.02,596842.6,715.5177,1,0-20%
4,14000US37141920403,Pender,9204.03,596842.6,715.5177,1,0-20%
5,14000US37141920601,Pender,9206.01,596842.6,715.5177,1,0-20%
6,14000US37141990100,Pender,9901.00,596842.6,715.5177,1,0-20%


In [3]:
# seeing if the geo ids are consistent in each file
length(unique(acs_df$GEO_ID))
length(unique(wildfire_hazard_df$GEO_ID))

[1] 2195

[1] 2189

In [4]:
# they're not so what geo ids aren't consistent?
`%notin%` <- Negate(`%in%`)
ids_df = acs_df %>%
    filter(GEO_ID %notin% unique(wildfire_hazard_df$GEO_ID)) %>%
    select(c("GEO_ID", "Census_Tract", "County")) 

# ids that aren't consistent
inconsistent_ids = ids_df$GEO_ID
inconsistent_ids

[1] "14000US37031990200" "14000US37055990200" "14000US37095990100"
[4] "14000US37095990200" "14000US37101041202" "14000US37137990100"

In [5]:
acs_df %>%
    filter(GEO_ID %in% inconsistent_ids)
wildfire_hazard_df %>%
    filter(GEO_ID %in% inconsistent_ids)

GEO_ID,Census_Tract,County,Sex_Ratio,Median_Age,Population_Under_18,Population_Over_18,Population_Over_65,Race_White,Race_Black,⋯,Poverty_Adult,Poverty_Elderly,Poverty_Male,Poverty_Female,Poverty_Black,Poverty_Asian,Poverty_Other_Race,Poverty_Mixed_Race,Poverty_Hispanic_Latino,Poverty_White
<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
14000US37031990200,9902.00,Carteret,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
14000US37055990200,9902.00,Dare,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
14000US37095990100,9901.00,Hyde,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
14000US37095990200,9902.00,Hyde,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
14000US37101041202,412.02,Johnston,92.8,36.5,26.8,73.2,12.9,81.9,12.7,⋯,19.9,7.5,18.4,19.9,16.1,0,0,69.7,31.1,13.7
14000US37137990100,9901.00,Pamlico,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


GEO_ID,County,Census_Tract,Avg_ACRES,Wildfire_Hazard_Potential_Mean,Wildfire_Hazard_Potential_Quintile_Number,Wildfire_Hazard_Potential_Quintile_Percentage
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>


Almost all ACS data is missing for these GEO ids that aren't in the WHP df, so they'll be removed.

In [6]:
acs_whp_df = inner_join(acs_df, wildfire_hazard_df)

head(acs_whp_df)

Joining, by = c("GEO_ID", "Census_Tract", "County")


,GEO_ID,Census_Tract,County,Sex_Ratio,Median_Age,Population_Under_18,Population_Over_18,Population_Over_65,Race_White,Race_Black,⋯,Poverty_Black,Poverty_Asian,Poverty_Other_Race,Poverty_Mixed_Race,Poverty_Hispanic_Latino,Poverty_White,Avg_ACRES,Wildfire_Hazard_Potential_Mean,Wildfire_Hazard_Potential_Quintile_Number,Wildfire_Hazard_Potential_Quintile_Percentage
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,14000US37001020100,201.00,Alamance,87.2,37.3,21.9,78.1,15.1,61.0,28.5,⋯,54.8,0,45.3,0.0,31.3,13.9,278233.8,92.23704,5,81-100
2,14000US37001020200,202.00,Alamance,78.3,31.0,29.1,70.9,11.2,51.9,29.2,⋯,44.6,0,0.0,82.7,16.2,39.0,278233.8,92.23704,5,81-100
3,14000US37001020300,203.00,Alamance,77.8,40.7,27.1,72.9,18.9,45.2,28.9,⋯,21.0,NA,10.9,36.4,26.7,20.1,278233.8,92.23704,5,81-100
4,14000US37001020400,204.00,Alamance,78.1,30.5,31.4,68.6,10.1,19.6,59.0,⋯,19.9,NA,69.0,100.0,67.2,26.6,278233.8,92.23704,5,81-100
5,14000US37001020501,205.01,Alamance,92.7,35.1,23.3,76.7,15.6,76.1,13.3,⋯,18.9,0,0.0,53.1,25.7,10.0,278233.8,92.23704,5,81-100
6,14000US37001020502,205.02,Alamance,99.0,32.1,30.0,70.0,8.9,40.7,35.7,⋯,27.8,100,14.2,19.5,14.7,25.0,278233.8,92.23704,5,81-100


# Variable Filtering
RF imputation needs at least 25% of data in order to impute the missing data, therefore variables with less than 25% won't be imputed. 

In [7]:
# only keeping variables with at least 25% of data in each county
variable_presence_df = acs_whp_df[,1:31] %>% 
    pivot_longer(cols = 4:31, names_to = "Variable", values_to = "Value") %>%
    # creating 2 count cols: 1st for non-missing data, 2nd for all data
    mutate(data_count = ifelse(is.na(Value), 0, 1), all_count = 1) %>%
    group_by(Variable) %>%
    summarize(Variable_Presence = sum(data_count)/ sum(all_count) * 100) %>%
    arrange(Variable_Presence)

head(variable_presence_df)

Variable,Variable_Presence
<chr>,<dbl>
Poverty_Asian,71.91781
Poverty_Other_Race,79.86301
Poverty_Mixed_Race,92.92237
Poverty_Black,94.79452
Poverty_Hispanic_Latino,96.80365
Poverty_Elderly,98.35616


All variables will be kept. 

# Random Forest Imputation
Generating missing values using random forest (RF).

In [8]:
# imputing sociodemographic data using RF
random_forest_imputation = function(preimputed_df){

    imputed_RF_object = missForest(as.matrix(preimputed_df[,4:30]))
    imputed_RF_df = imputed_RF_object$ximp
    imputed_values_df = data.frame(cbind(preimputed_df[,c(1:3,31:34)], imputed_RF_df)) 

    
    return(imputed_values_df)
}

# calling function
imputed_df = random_forest_imputation(acs_whp_df)
head(imputed_df)

,GEO_ID,Census_Tract,County,Avg_ACRES,Wildfire_Hazard_Potential_Mean,Wildfire_Hazard_Potential_Quintile_Number,Wildfire_Hazard_Potential_Quintile_Percentage,Sex_Ratio,Median_Age,Population_Under_18,⋯,Poverty_Adult,Poverty_Elderly,Poverty_Male,Poverty_Female,Poverty_Black,Poverty_Asian,Poverty_Other_Race,Poverty_Mixed_Race,Poverty_Hispanic_Latino,Poverty_White
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,14000US37001020100,201.00,Alamance,278233.8,92.23704,5,81-100,87.2,37.3,21.9,⋯,20.8,14.6,26.4,28.0,54.8,0.000,45.3,0.0,31.3,13.9
2,14000US37001020200,202.00,Alamance,278233.8,92.23704,5,81-100,78.3,31.0,29.1,⋯,33.0,19.2,27.3,37.8,44.6,0.000,0.0,82.7,16.2,39.0
3,14000US37001020300,203.00,Alamance,278233.8,92.23704,5,81-100,77.8,40.7,27.1,⋯,17.3,22.0,18.0,28.3,21.0,32.061,10.9,36.4,26.7,20.1
4,14000US37001020400,204.00,Alamance,278233.8,92.23704,5,81-100,78.1,30.5,31.4,⋯,26.8,14.0,30.0,33.3,19.9,31.430,69.0,100.0,67.2,26.6
5,14000US37001020501,205.01,Alamance,278233.8,92.23704,5,81-100,92.7,35.1,23.3,⋯,12.2,7.1,12.3,14.8,18.9,0.000,0.0,53.1,25.7,10.0
6,14000US37001020502,205.02,Alamance,278233.8,92.23704,5,81-100,99.0,32.1,30.0,⋯,22.3,19.2,17.7,31.0,27.8,100.000,14.2,19.5,14.7,25.0


In [9]:
# exporting data
write.csv(imputed_df, paste0(Output,"/", cur_date, "_Imputed_ACS_WHP_Data.csv"), row.names = FALSE)